# 220115 SVM Classfication model
### - GIS Partial Discharge Signal: Noise , Surface PD, Particle PD
### - Sensor: PD SENSOR

## 1. Python Module

In [1]:
# 경로 설정 패키지
import os

# 데이터 전처리 패키지
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler #정규화 패키지

# 기계학습 모델 및 평가 패키지
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.model_selection import KFold
from scipy.stats import skew, kurtosis
from tqdm import tqdm

# 시각화 패키지
import seaborn as sns 
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
from matplotlib.colors import ListedColormap
matplotlib.rcParams['axes.unicode_minus'] = False

#한글 폰트 설정 
plt.rc('font', family='Malgun Gothic')

## 2. Data Pre-processing

In [2]:
# Noise_data

os.chdir("C:/Users/taeyu/PRPD data Test/data/noise") #데이터 경로
print(os.getcwd())

data_name = []

for i in range(30):
    data = 'noise ({0}).csv'.format(i)
    data_name.append(data)

    
    
for j, k in zip(range(0,50),data_name):    
    df = pd.read_csv(k, header=None, index_col = False) # Dataframe 데이터
    test_data= df.to_numpy() # ndarray data
    
    feature_file = np.ones((3600,10), dtype=None, order='C')
    feature_file_mean = np.ones((1,11), dtype=None, order='C')
    
    for a in (range(3600)):
        feature_file[a,0] = skew(test_data[a,0:63]); # 60hZ 반주기 왜도(Positive)
        feature_file[a,1] = skew(test_data[a,64:127]); # 60hZ 반주기 왜도(Negaitive)
        feature_file[a,2] = kurtosis(test_data[a,0:63]); # 60hZ 반주기 첨도(Positive)
        feature_file[a,3] = kurtosis(test_data[a,64:127]); # 60hZ 반주기 첨도(Negaitive)
        feature_file[a,4] = np.max(test_data[a,0:63]); # 60hZ 반주기 최대값(Positive)
        feature_file[a,5] = np.max(test_data[a,64:127]); # 60hZ 반주기 최대값(Negaitive)
        feature_file[a,6] = np.mean(test_data[a,0:63]); # 60hZ 반주기 평균(Positive)
        feature_file[a,7] = np.mean(test_data[a,64:127]); # 60hZ 반주기 평균(Negaitive)
        feature_file[a,8] = np.count_nonzero(test_data[a,0:63]>40); # 60hZ 반주기 PD 신호 개수, threshhold 수정 
        feature_file[a,9] = np.count_nonzero(test_data[a,64:127]>40); # 60hZ 반주기 PD 신호 개수, threshhold 수정 
        
        feature_file_mean[0,0] = np.mean(feature_file[:,0]); #3600 사이클 평균 
        feature_file_mean[0,1] = np.mean(feature_file[:,1]); 
        feature_file_mean[0,2] = np.mean(feature_file[:,2]);
        feature_file_mean[0,3] = np.mean(feature_file[:,3]);
        feature_file_mean[0,4] = np.mean(feature_file[:,4]);
        feature_file_mean[0,5] = np.mean(feature_file[:,5]);
        feature_file_mean[0,6] = np.mean(feature_file[:,6]);
        feature_file_mean[0,7] = np.mean(feature_file[:,7]);
        feature_file_mean[0,8] = np.mean(feature_file[:,8]);
        feature_file_mean[0,9] = np.mean(feature_file[:,9]);
        feature_file_mean[0,10] = 0 # Noise 라벨링 
        
    globals()['prpd_{}'.format(j)] = pd.DataFrame(feature_file_mean);


df_noise = pd.concat([prpd_0, prpd_1, prpd_2, prpd_3, prpd_4, prpd_5, prpd_6, prpd_7, prpd_8, prpd_9, prpd_10, prpd_11,
                         prpd_12, prpd_13,  prpd_14, prpd_15 , prpd_16,  prpd_17, prpd_18, prpd_19, prpd_20,
                         prpd_21 , prpd_22,prpd_23,  prpd_24 , prpd_25, prpd_26,  prpd_27,  prpd_28, prpd_29])

C:\Users\taeyu\PRPD data Test\data\noise


In [3]:
# Surface_data

os.chdir("C:/Users/taeyu/PRPD data Test/data/surface") 
print(os.getcwd())

data_name = []

for i in range(30):
    data = 'surface ({0}).csv'.format(i)
    data_name.append(data)

    
    
for j, k in zip(range(0,50),data_name):    
    df = pd.read_csv(k, header=None, index_col = False) # Dataframe 데이터
    test_data= df.to_numpy() # ndarray data
    
    feature_file = np.ones((3600,10), dtype=None, order='C')
    feature_file_mean = np.ones((1,11), dtype=None, order='C')
    
    for a in (range(3600)):
        feature_file[a,0] = skew(test_data[a,0:63]);
        feature_file[a,1] = skew(test_data[a,64:127]);
        feature_file[a,2] = kurtosis(test_data[a,0:63]);
        feature_file[a,3] = kurtosis(test_data[a,64:127]);
        feature_file[a,4] = np.max(test_data[a,0:63]);
        feature_file[a,5] = np.max(test_data[a,64:127]);
        feature_file[a,6] = np.mean(test_data[a,0:63]);
        feature_file[a,7] = np.mean(test_data[a,64:127]);
        feature_file[a,8] = np.count_nonzero(test_data[a,0:63]>40); #  threshhold 수정
        feature_file[a,9] = np.count_nonzero(test_data[a,64:127]>40); #  threshhold 수정
        
        feature_file_mean[0,0] = np.mean(feature_file[:,0]);
        feature_file_mean[0,1] = np.mean(feature_file[:,1]);
        feature_file_mean[0,2] = np.mean(feature_file[:,2]);
        feature_file_mean[0,3] = np.mean(feature_file[:,3]);
        feature_file_mean[0,4] = np.mean(feature_file[:,4]);
        feature_file_mean[0,5] = np.mean(feature_file[:,5]);
        feature_file_mean[0,6] = np.mean(feature_file[:,6]);
        feature_file_mean[0,7] = np.mean(feature_file[:,7]);
        feature_file_mean[0,8] = np.mean(feature_file[:,8]);
        feature_file_mean[0,9] = np.mean(feature_file[:,9]);
        feature_file_mean[0,10] = 1 # Surface 라벨링              
        
    globals()['prpd_{}'.format(j)] = pd.DataFrame(feature_file_mean);


df_surface = pd.concat([prpd_0, prpd_1, prpd_2, prpd_3, prpd_4, prpd_5, prpd_6, prpd_7, prpd_8, prpd_9, prpd_10, prpd_11,
                         prpd_12, prpd_13,  prpd_14, prpd_15 , prpd_16,  prpd_17, prpd_18, prpd_19, prpd_20,
                         prpd_21 , prpd_22,prpd_23,  prpd_24 , prpd_25, prpd_26,  prpd_27,  prpd_28, prpd_29,  prpd_30]) #수정 

C:\Users\taeyu\PRPD data Test\data\surface


NameError: name 'prpd_30' is not defined

In [ ]:
# Particle_data

os.chdir("C:/Users/taeyu/PRPD data Test/data/particle") #수정 
print(os.getcwd())

data_name = []

for i in range(30):
    data = 'particle ({0}).csv'.format(i) #수정 
    data_name.append(data)

    
    
for j, k in zip(range(0,50),data_name):    
    df = pd.read_csv(k, header=None, index_col = False) # Dataframe 데이터
    test_data= df.to_numpy() # ndarray data
    
    feature_file = np.ones((3600,10), dtype=None, order='C')
    feature_file_mean = np.ones((1,11), dtype=None, order='C')
    
    for a in (range(3600)):
        feature_file[a,0] = skew(test_data[a,0:63]);
        feature_file[a,1] = skew(test_data[a,64:127]);
        feature_file[a,2] = kurtosis(test_data[a,0:63]);
        feature_file[a,3] = kurtosis(test_data[a,64:127]);
        feature_file[a,4] = np.max(test_data[a,0:63]);
        feature_file[a,5] = np.max(test_data[a,64:127]);
        feature_file[a,6] = np.mean(test_data[a,0:63]);
        feature_file[a,7] = np.mean(test_data[a,64:127]);
        feature_file[a,8] = np.count_nonzero(test_data[a,0:63]>40); #  threshhold 수정
        feature_file[a,9] = np.count_nonzero(test_data[a,64:127]>40); #  threshhold 수정
        
        feature_file_mean[0,0] = np.mean(feature_file[:,0]);
        feature_file_mean[0,1] = np.mean(feature_file[:,1]);
        feature_file_mean[0,2] = np.mean(feature_file[:,2]);
        feature_file_mean[0,3] = np.mean(feature_file[:,3]);
        feature_file_mean[0,4] = np.mean(feature_file[:,4]);
        feature_file_mean[0,5] = np.mean(feature_file[:,5]);
        feature_file_mean[0,6] = np.mean(feature_file[:,6]);
        feature_file_mean[0,7] = np.mean(feature_file[:,7]);
        feature_file_mean[0,8] = np.mean(feature_file[:,8]);
        feature_file_mean[0,9] = np.mean(feature_file[:,9]);
        feature_file_mean[0,10] = 2 # Particle 라벨링        수정필요
        
    globals()['prpd_{}'.format(j)] = pd.DataFrame(feature_file_mean);


df_particle = pd.concat([prpd_0, prpd_1, prpd_2, prpd_3, prpd_4, prpd_5, prpd_6, prpd_7, prpd_8, prpd_9, prpd_10, prpd_11,
                         prpd_12, prpd_13,  prpd_14, prpd_15 , prpd_16,  prpd_17, prpd_18, prpd_19, prpd_20,
                         prpd_21 , prpd_22,prpd_23,  prpd_24 , prpd_25, prpd_26,  prpd_27,  prpd_28, prpd_29,  prpd_30]) #수정 

In [ ]:
df_particle.head() # 데이터 확인

In [ ]:
df_noise.head() # 데이터 확인

In [ ]:
df_surface.head() # 데이터 확인

In [ ]:
final_data = pd.concat([df_noise, df_surface, df_particle])

os.chdir("C:/Users/taeyu/PRPD data Test") #수정 
print(os.getcwd())

final_data.to_csv('prpd_final.csv', index=False, header=None) 

## 3. 데이터 불러오기(다변랑 데이터)

In [ ]:
df = pd.read_csv('prpd_final.csv', header=None, index_col = False) # Dataframe 데이터
df.head(150)

In [ ]:
df.describe() # 통계량값 확인.

In [ ]:
data = df.to_numpy() # ndarray data
print(data)  # 데이터 인덱싱 

In [ ]:
X = data[:,0:10] # 독립 변수(feature)
Y = data[:,10] # 종속 변수 (target)

## 4. 데이터 분리(학습 8: 테스트 2) 

In [ ]:
from sklearn.model_selection import train_test_split 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2022) 

## 5. 데이터 정규화

In [ ]:
# Scaling

normalizer = StandardScaler()
X_train = normalizer.fit_transform(X_train)
X_test = normalizer.transform(X_test)


## 6. SVM 학습

In [ ]:
# SVM 학습 (Best 조건)
from sklearn.svm import SVC

classifier = SVC(kernel='rbf', gamma=0.01, C=1)
classifier.fit(X_train, Y_train)

classifier.score(X_test,Y_test)

## DecisionTreeClassifier() 트리모델
## RandomForestClassifier() 랜덤포레스트
## KNeighborsClassifier() K-최접근 
## SVC(kernel="linear"), 서포터 벡터 머신 
## GaussianNB(), 가우시안 NB
## LogisticRegression(), 로지스트 회귀 

## 7. 학습결과

In [ ]:
# train, test acc

Y_train_pred = classifier.predict(X_train)
Y_test_pred = classifier.predict(X_test)

acc_train = accuracy_score(Y_train, Y_train_pred)
acc_test = accuracy_score(Y_test, Y_test_pred)

print('Training Accuracy: {:3f}'. format(acc_train))
print('Testing Accuracy: {:3f}'. format(acc_test))

In [ ]:
# 혼돈 행렬 (대각선 이외의 값은 예측에 실패한 개수를 표현하기 때문에 대각선 값이 크게 나타나야지 좋다고 볼수 있음, 테스트에 90개 사용)
plot_confusion_matrix(classifier, X_test, Y_test)
plt.show()

## 8. 최적화 및 교차 검증(GridSearchCV)

In [ ]:
# gridsearch 

from sklearn.model_selection import GridSearchCV
  
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}
  
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose= 3)

  
# fitting the model for grid search
grid.fit(X_train, Y_train);

# print best parameter after tuning
print(grid.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)


In [ ]:
grid.best_params_